<a href="https://colab.research.google.com/github/Nunez350/DataScienceAndAnalytics/blob/main/Module_11_Optimization_pt3/retail_pricing_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Analytics Edge of Retail Optimization

Both binary variables and non-linear functions


#gurobipy
The Gurobi Optimizer is a mathematical optimization software library for solving mixed-integer linear and quadratic optimization problems.

In [4]:
!pip install gurobipy

# load require packages

In [5]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import plotly.express as px

### Lists

In [ ]:
# Be careful with Python indices!
# The range function range(a,b) creates a range of integers starting at <a> but ending at <b-1>
weeks = list(range(157,170))
print("weeks (t) ", weeks)
# Also note here that the first element of a list has index 0. The second element has index 1, and so on...
w1 = weeks[0] #denotes the number of the first week in our planning horizon
print(w1)
w2 = weeks[1] #denotes the number of the second week in our planning horizon
print(w2)

weeks (t)  [157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169]
157
158


### Parameters

In [ ]:
#From orginal price / sales(demand) data
# Developed linear demand model: used 27-156 weeks (3 years) of data; 27-104( years 1/2)Train, 105-156( last year)Test

# Variable: 𝑆𝑒𝑎𝑠𝑜𝑛X. Year divided int 13 seasons / 4 weeks per season (categorical variable)
# Variable: Price,  at time 𝑝_𝑡, 𝑝_(𝑡−1), 𝑝_(𝑡−2)
# Linear regression developed demand model (Constrain 1)
# Demand is characterized by our linearly additive model
# model: 𝑑_𝑡 = 2181 − 2801*𝑝_𝑡 + 929*𝑝_(𝑡−1) + 728*𝑝_(𝑡−2) −555.430*𝑆𝑒𝑎𝑠𝑜𝑛2 +...+ 949.056*𝑆𝑒𝑎𝑠𝑜𝑛13

# Store model parameters
intercept = 2181
# causal factors
p_coeff = -2801
p1_coeff = 929
p2_coeff = 728
# time series factor
season_coeff = {1: 0, 2: -555.430, 3: 81.746,  4: -406.774, 5: -26.122, 6: -8.292,
               7: -39.334, 8: -81.407, 9: 148.728, 10: 1107.254, 11: 1125.259, 12: 1171.240, 13: 949.056}

In [ ]:
# Here we create a dictionary that associates a season with each week in the planning horizon
season = {}
for w in weeks:
    season[w] = np.ceil((w % 52) / 4)

print(season)

{157: 1.0, 158: 1.0, 159: 1.0, 160: 1.0, 161: 2.0, 162: 2.0, 163: 2.0, 164: 2.0, 165: 3.0, 166: 3.0, 167: 3.0, 168: 3.0, 169: 4.0}


## Model 1 - no price ladder or business rules
#### DECISION VARIABLES:  prices in weeks 157 – 169  and demand in weeks 157 – 169
######  Prices; 𝑝_𝑡  for  t = 157, … , 169   and   𝑑_𝑡  for  t = 157, … , 169
######  Demand; follows linearly additive model:
#### MAXIMIZE 𝑅𝑒𝑣𝑒𝑛𝑢𝑒= 𝑝157𝑑157 + 𝑝158𝑑158 + 𝑝159𝑑159 + ... + 𝑝169𝑑169
#### Objective function,  sales revenue in next quarter (weeks 157 – 169)

1.   List item
2.   List item


#### SUBJECT TO CONSTRAINTS:
#### Demand is characterized by our linearly additive model:
#### 𝑑_𝑡 = 2181 − 2801*𝑝_𝑡 + 929*𝑝_(𝑡−1) + 728*𝑝_(𝑡−2) −555.430*𝑆𝑒𝑎𝑠𝑜𝑛2 +...+ 949.056*𝑆𝑒𝑎𝑠𝑜𝑛13


In [ ]:
# Create Gurobi model object - repository for all objects to be used in the model
mod1 = gp.Model ("price_model_1")

Restricted license - for non-production use only - expires 2023-10-25


In [ ]:
# Define decision variables
p = mod1.addVars(weeks, ub = 1)

In [ ]:
p

{157: <gurobi.Var *Awaiting Model Update*>,
 158: <gurobi.Var *Awaiting Model Update*>,
 159: <gurobi.Var *Awaiting Model Update*>,
 160: <gurobi.Var *Awaiting Model Update*>,
 161: <gurobi.Var *Awaiting Model Update*>,
 162: <gurobi.Var *Awaiting Model Update*>,
 163: <gurobi.Var *Awaiting Model Update*>,
 164: <gurobi.Var *Awaiting Model Update*>,
 165: <gurobi.Var *Awaiting Model Update*>,
 166: <gurobi.Var *Awaiting Model Update*>,
 167: <gurobi.Var *Awaiting Model Update*>,
 168: <gurobi.Var *Awaiting Model Update*>,
 169: <gurobi.Var *Awaiting Model Update*>}

In [ ]:
# Set objective function
# Example first 2 lines fully written out,
# Short cut for writing out each line
#  after the first 2 weeks, the remaining weeks[2:], are developed using the sum() and a for loop  (weeks 159 on...)
#    price at week(wx) * demand (from linear demand fucntion at weekx)
obj_fn = mod1.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [ ]:
mod1.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 0 rows, 13 columns and 0 nonzeros
Model fingerprint: 0x7ae80c7d
Model has 36 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [2e+03, 4e+03]
  QObjective range [1e+03, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.02s
Presolved: 0 rows, 13 columns, 0 nonzeros
Presolved model has 36 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 12
 AA' NZ     : 6.600e+01
 Factor NZ  : 7.800e+01
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.60181469e+07  3.08140123e+05  9.99e+02 2.49e+02  1.00e+06     0s
   1   3.71234768e+04  1.33560572e+05  9.60e-01 2.40e

In [ ]:
p[157] # to see 1st prediction

<gurobi.Var C0 (value 0.9621363090365429)>

In [ ]:
p[158] # to see 2nd prediction

<gurobi.Var C1 (value 0.948021817562962)>

In [ ]:
# to see all predictions and save to a dataframe
df1 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df1.loc[w,"price"] = p[w].x
df1

,price
157,0.962136
158,0.948022
159,0.921944
160,0.894928
161,0.784609
162,0.764887
163,0.764654
164,0.780981
165,0.89384
166,0.895477


In [ ]:
fig = px.line(df1, x=df1.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 2 - no business rules + price ladder

In [ ]:
p_ladder = [1.0, 0.9, 0.8, 0.7] #create a list for price ladder

In [ ]:
# Create model object
mod2 = gp.Model ("price_model_2")

In [ ]:
# Define decision variables which includes ladder constraint
p = mod2.addVars(weeks)
x = mod2.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [ ]:
# Set objective function
obj_fn = mod2.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [ ]:
# Select price ladder value
constr_select_ladder = mod2.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod2.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)

In [ ]:
mod2.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 26 rows, 65 columns and 117 nonzeros
Model fingerprint: 0x5eb5a9f6
Model has 36 quadratic objective terms
Variable types: 13 continuous, 52 integer (52 binary)
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [2e+03, 4e+03]
  QObjective range [1e+03, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 11514.818400
Presolve removed 0 rows and 13 columns
Presolve time: 0.00s
Presolved: 26 rows, 52 columns, 91 nonzeros
Presolved model has 36 quadratic objective terms
Variable types: 13 continuous, 39 integer (39 binary)

Root relaxation: objective 1.201770e+04, 37 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node T

In [ ]:
df2 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df2.loc[w,"price"] = p[w].x
df2

,price
157,1.0
158,1.0
159,0.9
160,0.9
161,0.8
162,0.8
163,0.8
164,0.8
165,0.9
166,0.9


In [ ]:
fig = px.line(df2, x=df2.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 3 - at most 4 promotions

In [ ]:
# Create model object
mod3 = gp.Model ("price_model_3")

In [ ]:
# Define decision variables
p = mod3.addVars(weeks)
x = mod3.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [ ]:
# Set objective function
obj_fn = mod3.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [ ]:
# Select price ladder value
constr_select_ladder = mod3.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod3.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)
# At most 4 promotions
constr_4_promo = mod3.addConstr(sum(x[w,k] for k in p_ladder[1:] for w in weeks) <= 4)

In [ ]:
mod3.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 27 rows, 65 columns and 156 nonzeros
Model fingerprint: 0x8356128d
Model has 36 quadratic objective terms
Variable types: 13 continuous, 52 integer (52 binary)
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [2e+03, 4e+03]
  QObjective range [1e+03, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 10890.440000
Presolve time: 0.00s
Presolved: 27 rows, 65 columns, 143 nonzeros
Presolved model has 36 quadratic objective terms
Variable types: 13 continuous, 52 integer (52 binary)

Root relaxation: objective 1.194927e+04, 65 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 11949.2697    0   20

In [ ]:
df3 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df3.loc[w,"price"] = p[w].x
df3

,price
157,1.0
158,1.0
159,1.0
160,1.0
161,1.0
162,1.0
163,0.9
164,0.9
165,1.0
166,1.0


In [ ]:
fig = px.line(df3, x=df3.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 4 - full model

In [ ]:
# Create model object
mod4 = gp.Model ("price_model_4")

In [ ]:
# Define decision variables
p = mod4.addVars(weeks)
x = mod4.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [ ]:
# Set objective function
obj_fn = mod4.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [ ]:
# Select price ladder value
constr_select_ladder = mod4.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod4.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)
# At most 4 promotions
constr_4_promo = mod4.addConstr(sum(x[w,k] for k in p_ladder[1:] for w in weeks) <= 4)
# No consecutive promotions
constr_no_consec_promo = mod4.addConstrs((sum(x[w,k] for k in p_ladder[1:]) + sum(x[w+1,k] for k in p_ladder[1:]) <= 1) for w in weeks[:-1])

In [ ]:
mod4.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 39 rows, 65 columns and 228 nonzeros
Model fingerprint: 0xf7b2a2b1
Model has 36 quadratic objective terms
Variable types: 13 continuous, 52 integer (52 binary)
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [2e+03, 4e+03]
  QObjective range [1e+03, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 11126.462000
Presolve time: 0.00s
Presolved: 39 rows, 65 columns, 215 nonzeros
Presolved model has 36 quadratic objective terms
Variable types: 13 continuous, 52 integer (52 binary)

Root relaxation: objective 1.187440e+04, 73 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 11874.4019    0   26

In [ ]:
df4 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df4.loc[w,"price"] = p[w].x
df4

,price
157,1.0
158,1.0
159,1.0
160,1.0
161,1.0
162,0.9
163,1.0
164,0.9
165,1.0
166,1.0


In [ ]:
fig = px.line(df4, x=df4.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

### End of script
---


In [6]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import plotly.express as px

# Define the weeks for the planning horizon
weeks = list(range(157, 164))

# Model parameters
intercept = 1.978242858
p_coeff = -2.809634145
p1_coeff = 0.963410728
p2_coeff = 0.759639170
season_coeff = {2: -0.562046910, 3: 0.087545274, 4: -0.402637480, 5: -0.027326010,
                6: 0.004349885, 7: -0.036102297, 8: -0.069280527, 9: 0.160276197,
                10: 1.104208897, 11: 1.122711287, 12: 1.176802194, 13: 0.947945548}

#
# Define the correct price ladder
price_ladder = [1.00, 0.95, 0.85, 0.75, 0.60, 0.50]

# Create a dictionary to associate a season with each week
season = {w: int(np.ceil((w % 52) / 4)) for w in weeks}

In [7]:
# Initialize Gurobi model
mod1 = gp.Model("price_model_with_ladder")

# Define binary decision variables for price selection
price_vars = mod1.addVars(weeks, price_ladder, vtype=GRB.BINARY, name="price_choice")

# Define demand variables
d = mod1.addVars(weeks, vtype=GRB.CONTINUOUS, name="demand")

# Ensure only one price is chosen per week
for w in weeks:
    mod1.addConstr(gp.quicksum(price_vars[w, p] for p in price_ladder) == 1)

Restricted license - for non-production use only - expires 2025-11-24


In [8]:
# Define the objective function: Maximize total revenue
mod1.setObjective(gp.quicksum(price_vars[w, p] * p * d[w] for w in weeks for p in price_ladder), GRB.MAXIMIZE)


In [9]:
# Define the constraints based on the updated demand model
for t in weeks:
    season_effect = sum(season_coeff[s] if season[t] == s else 0 for s in season_coeff.keys())
    if t == 157:
        mod1.addConstr(
            d[t] == intercept +
            gp.quicksum(price_vars[t, p] * p_coeff * p for p in price_ladder) +
            p1_coeff * 1 +
            p2_coeff * 1 +
            season_effect)
    elif t == 158:
        mod1.addConstr(
            d[t] == intercept +
            gp.quicksum(price_vars[t, p] * p_coeff * p for p in price_ladder) +
            gp.quicksum(price_vars[t-1, p] * p1_coeff * p for p in price_ladder) +
            p2_coeff * 1 +
            season_effect)
    else:
        mod1.addConstr(
            d[t] == intercept +
            gp.quicksum(price_vars[t, p] * p_coeff * p for p in price_ladder) +
            gp.quicksum(price_vars[t-1, p] * p1_coeff * p for p in price_ladder) +
            gp.quicksum(price_vars[t-2, p] * p2_coeff * p for p in price_ladder) +
            season_effect)


In [10]:
# Optimize the model
mod1.optimize()

# Extract the optimal prices and demands
prices = {w: sum(p * price_vars[w, p].x for p in price_ladder) for w in weeks}
demands = {w: d[w].x for w in weeks}

# Save the results to a DataFrame
df1 = pd.DataFrame({'price': prices, 'demand': demands})

# Plot the prices
fig = px.line(df1, x=df1.index, y='price', markers=True)
fig.update_layout(plot_bgcolor="white", xaxis_title="Week")
fig.update_traces(line_color="red")
fig.show()


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 14 rows, 49 columns and 157 nonzeros
Model fingerprint: 0xff10d7a9
Model has 42 quadratic objective terms
Variable types: 7 continuous, 42 integer (42 binary)
Coefficient statistics:
  Matrix range     [4e-01, 3e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve time: 0.00s
Presolved: 56 rows, 91 columns, 265 nonzeros
Variable types: 49 continuous, 42 integer (42 binary)
Found heuristic solution: objective 4.5554695
Found heuristic solution: objective 4.7961861
Found heuristic solution: objective 4.7999295

Root relaxation: objective 1.193299e+01, 44 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Obj